# Importation des packages et connexion au S3

In [6]:
from helpers import *
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
pd.set_option('display.float_format', '{:.2f}'.format)
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_white


In [7]:
s3=s3_connection()

connection successful


# 1. La motivation

In [8]:
regional_stat=s3.get_tables_from_s3("clichere/diffusion/final_comm.parquet")
print(regional_stat[["pop2022", "revmoy2022", "psup2022", "pcadr2022"]].describe())

PermissionError: Forbidden

In [9]:
df=s3.get_tables_from_s3("aayrapetyan/diffusion/final_table.parquet")
print(df[["pop2022", "revmoy2022", "psup2022", "pcadr2022"]].describe())

PermissionError: Forbidden

On voit que la population moyenne diminue lorsque l'on sélectionne les communes ayant une gare. Cela correspond au fait que les petites communes ont moins de chance d'avoir une gare. On passe de 36000 communes à 2800 communes, ce qui donne tout de même beaucoup d'observations. On remarque que le revenu moyen d'une commune en France est plus élevé lorsque l'on sélectionne seulement les communes comprenant une gare. De même, les personnes sont en moyenne plus diplômée et appartiennent aux catégories socio-professionnelles plus élevées également. 

On veut montrer qu'il existe un lien entre le revenu, la catégorie socio-professionnelle (csp) et le fait de prendre un train en France. Pour cela, on effectue une régression linéaire avec le nombre total de voyageurs comme variable dépendante. Notre but est d'expliquer pourquoi est-ce que cette variable est différente en fonction des communes, en contrôlant par la population de la commune. Les premières visualisations et les premières données suggèrent qu'il peut y avoir un lien entre son revenu et le fait de prendre un train.

In [25]:
print(df.columns)

Index(['total_voyageurs_2022', 'nom_gare', 'uic', 'codeinsee', 'lon_gare', 'lat_gare', 'codecommune', 'nomcommune', 'pop2022', 'revmoy2022', 'pbac2022', 'psup2022', 'pagri2022', 'pindp2022', 'pcadr2022', 'ppint2022', 'pempl2022', 'pouvr2022', 'pchom2022', 'dist_closest_station_km', 'regions'], dtype='object')


# 2. Régressions

In [29]:
dummies = pd.get_dummies(df['regions'], prefix=None)
dummies = dummies.astype(int)

df = pd.concat([df, dummies], axis=1)
print(df)

      total_voyageurs_2022               nom_gare       uic codeinsee  lon_gare  lat_gare codecommune             nomcommune  pop2022  revmoy2022  pbac2022  psup2022  pagri2022  pindp2022  pcadr2022  ppint2022  pempl2022  pouvr2022  pchom2022  dist_closest_station_km                     regions  Auvergne-Rhône-Alpes  Bourgogne-Franche-Comté  Bretagne  Centre-Val de Loire  Grand Est  Hauts-de-France  Normandie  Nouvelle-Aquitaine  Occitanie  Pays de la Loire  Provence-Alpes-Côte d'Azur  Île-de-France
0                    40825               Abbaretz  87481614     44001     -1.52     47.55       44001               ABBARETZ  2257.00    12567.25      0.27      0.17       0.06       0.02       0.05       0.14       0.31       0.42       0.04                     9.75            Pays de la Loire                     0                        0         0                    0          0                0          0                   0          0                 1                           0      

In [31]:
print(df.columns)

Index(['total_voyageurs_2022', 'nom_gare', 'uic', 'codeinsee', 'lon_gare', 'lat_gare', 'codecommune', 'nomcommune', 'pop2022', 'revmoy2022', 'pbac2022', 'psup2022', 'pagri2022', 'pindp2022', 'pcadr2022', 'ppint2022', 'pempl2022', 'pouvr2022', 'pchom2022', 'dist_closest_station_km', 'regions', 'Auvergne-Rhône-Alpes', 'Bourgogne-Franche-Comté', 'Bretagne', 'Centre-Val de Loire', 'Grand Est', 'Hauts-de-France', 'Normandie', 'Nouvelle-Aquitaine', 'Occitanie', 'Pays de la Loire', 'Provence-Alpes-Côte d'Azur', 'Île-de-France', 'pnodip2022'], dtype='object')


In [30]:
df['pnodip2022']=1-df['pbac2022']
df[['pbac2022', 'psup2022', 'pnodip2022', 'pagri2022', 'pempl2022', 'pcadr2022', 'pchom2022', 'pindp2022', 'pouvr2022', 'ppint2022']]=df[['pbac2022', 'psup2022', 'pnodip2022', 'pagri2022', 'pempl2022', 'pcadr2022', 'pchom2022', 'pindp2022', 'pouvr2022', 'ppint2022']]*100

In [34]:

X = df[['revmoy2022', 'pop2022', 'pnodip2022', 'psup2022', 'pcadr2022', 'pouvr2022', 'pchom2022', 'pempl2022','ppint2022', 'pagri2022', 'dist_closest_station_km', 'Auvergne-Rhône-Alpes', 'Bourgogne-Franche-Comté', 'Bretagne', 'Centre-Val de Loire', 'Grand Est', 'Hauts-de-France', 'Normandie', 'Nouvelle-Aquitaine', 'Occitanie', 'Pays de la Loire', 'Provence-Alpes-Côte d\'Azur', 'Île-de-France']]
Y = df['total_voyageurs_2022']  

X = sm.add_constant(X)

model0 = sm.OLS(Y, X).fit()

print(model0.summary())

                             OLS Regression Results                             
Dep. Variable:     total_voyageurs_2022   R-squared:                       0.215
Model:                              OLS   Adj. R-squared:                  0.209
Method:                   Least Squares   F-statistic:                     34.65
Date:                  Mon, 23 Dec 2024   Prob (F-statistic):          7.90e-129
Time:                          16:10:25   Log-Likelihood:                -45685.
No. Observations:                  2808   AIC:                         9.142e+04
Df Residuals:                      2785   BIC:                         9.155e+04
Df Model:                            22                                         
Covariance Type:              nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------


In [35]:
summary0_df = model0.summary2().tables[1]
print(summary0_df)

                                 Coef.   Std.Err.     t  P>|t|      [0.025     0.975]
const                      -2676070.24 1089217.99 -2.46   0.01 -4811826.46 -540314.01
revmoy2022                       79.11      10.71  7.39   0.00       58.12     100.11
pop2022                          11.71       1.19  9.88   0.00        9.38      14.03
pnodip2022                    12007.59    9689.36  1.24   0.22    -6991.46   31006.64
psup2022                      20469.30   10900.82  1.88   0.06     -905.20   41843.81
pcadr2022                     10305.05   11427.34  0.90   0.37   -12101.86   32711.96
pouvr2022                     13229.56    9858.55  1.34   0.18    -6101.25   32560.36
pchom2022                     22196.52   11617.83  1.91   0.06     -583.90   44976.94
pempl2022                      7165.89   10061.70  0.71   0.48   -12563.25   26895.03
ppint2022                     -9968.30   10129.76 -0.98   0.33   -29830.89    9894.28
pagri2022                      9294.78   15873.22  0.5

On enlève des csp les professions indépendantes qui regroupe des personnes très hétérogènes pour éviter la colinéarité parfaite. On obtient des coefficients significatifs sauf pour le taux de personnes sans diplôme dans la commune.

Essayons de passer au logarithme du nombre de voyageurs pour capturer des non-linéarités potentielles.

In [36]:
df['log_nb_voy2022']= np.log(df['total_voyageurs_2022'])

In [37]:
X = df[['revmoy2022', 'pop2022', 'psup2022', 'pnodip2022', 'pcadr2022', 'pouvr2022', 'pchom2022', 'pempl2022', 'pagri2022', 'ppint2022', 'dist_closest_station_km', 'Auvergne-Rhône-Alpes', 'Bourgogne-Franche-Comté', 'Bretagne', 'Centre-Val de Loire', 'Grand Est', 'Hauts-de-France', 'Normandie', 'Nouvelle-Aquitaine', 'Occitanie', 'Pays de la Loire', 'Provence-Alpes-Côte d\'Azur', 'Île-de-France']]
Y = df['log_nb_voy2022']  

X = sm.add_constant(X)

model1 = sm.OLS(Y, X).fit()

print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:         log_nb_voy2022   R-squared:                       0.358
Model:                            OLS   Adj. R-squared:                  0.353
Method:                 Least Squares   F-statistic:                     70.72
Date:                Mon, 23 Dec 2024   Prob (F-statistic):          1.07e-248
Time:                        16:12:13   Log-Likelihood:                -5934.5
No. Observations:                2808   AIC:                         1.191e+04
Df Residuals:                    2785   BIC:                         1.205e+04
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

In [38]:
summary1_df = model1.summary2().tables[1]
print(summary1_df)

                            Coef.  Std.Err.     t  P>|t|  [0.025  0.975]
const                        6.86      0.77  8.85   0.00    5.34    8.38
revmoy2022                  -0.00      0.00 -1.53   0.13   -0.00    0.00
pop2022                      0.00      0.00  9.01   0.00    0.00    0.00
psup2022                     0.02      0.01  2.62   0.01    0.01    0.04
pnodip2022                   0.01      0.01  1.47   0.14   -0.00    0.02
pcadr2022                    0.04      0.01  4.83   0.00    0.02    0.06
pouvr2022                    0.02      0.01  2.86   0.00    0.01    0.03
pchom2022                    0.05      0.01  5.51   0.00    0.03    0.06
pempl2022                    0.01      0.01  1.93   0.05   -0.00    0.03
pagri2022                   -0.07      0.01 -5.99   0.00   -0.09   -0.05
ppint2022                    0.01      0.01  1.68   0.09   -0.00    0.03
dist_closest_station_km      0.08      0.01  7.30   0.00    0.06    0.11
Auvergne-Rhône-Alpes         0.78      0.13  6.19  

On note que les coefficients sont significatifs sauf pour le revenu moyen des communes. Après avoir contrôlé pour les autres variables, et en ajustant le nombre de voyageur en logarithme, les effets du revenu ainsi que le fait de n'avoir pas de diplôme ne sont plus significatifs. En revanche, tous les autres coefficients le sont à 1% sauf la distance de la gare la plus proche, dont le coefficient est significatif à 5%. Notons que le R^2 ajusté se situe autour de 19% ce qui est assez élevé.

Nous allons 

In [ ]:
white_test = het_white(model1.resid, X)

# Résultats du test
stat, p_value, f_stat, f_p_value = white_test
print("Test de White:")
print(f"Statistique LM (Chi2): {stat:.4f}")
print(f"P-value LM: {p_value:.4f}")
print(f"Statistique F: {f_stat:.4f}")
print(f"P-value F: {f_p_value:.4f}")

# Interprétation
if p_value < 0.05:
    print("L'hypothèse d'homoskédasticité est rejetée. Il y a présence d'hétéroskédasticité.")
else:
    print("L'hypothèse d'homoskédasticité ne peut pas être rejetée.")

Augmenter d'un point de pourcentage la proportion de cadre dans la commune augmente de 0.07% le nombre de voyageur dans une gare. Le coefficient ne peut pas être interprété comme un effect causal dû à l'endogénéité du modèle. La densité de population peut être perçue comme un biais de variable oubliée.  

# 3. Conclusion

Nous avons montrer qu'il existe un lien entre le pourcentage de catégories socio-professionnels et le nombre de voyageurs dans les communes. Cela ne suffit pas à en déduire un lien causal dû à l'endogénéité du modèle. Cela ne nous donne pas un modèle causal mais un modèle de prédiction. Il faudrait ajouter certaines variables comme les variables de densité de population ou de types de transports utilisés. 